In [1]:
import pandas as pd

In [2]:
gameplay = pd.read_csv('User Gameplay data.csv')
withdraw = pd.read_csv('Withdrawal Data.csv')
deposit = pd.read_csv('Deposit Data.csv')

In [3]:
gameplay.set_index('Datetime', inplace=True)

In [4]:
withdraw.set_index('Datetime', inplace=True)

In [5]:
deposit.set_index('Datetime', inplace=True)

In [6]:
gameplay = gameplay.rename(columns={'User ID':'User Id'})

In [7]:
gameplay_data = gameplay.groupby('User Id').size().reset_index(name='no. of games played')

In [8]:
gameplay_data

,User Id,no. of games played
0,0,15
1,1,8
2,2,97
3,3,80
4,4,5
...,...,...
995,995,18
996,996,474
997,997,109
998,998,4


In [9]:
withdraw= withdraw.rename(columns={'Amount':'Withdraw Amount'})

In [10]:
num_times_withdraw = withdraw.groupby('User Id').size().reset_index(name='no. of times withdrawn')

In [11]:
total_withdraw = withdraw.groupby('User Id').agg({'Withdraw Amount': 'sum'}).reset_index()

In [12]:
withdraw_data =pd.merge(num_times_withdraw, total_withdraw, on='User Id', how='outer')

In [13]:
withdraw_data

,User Id,no. of times withdrawn,Withdraw Amount
0,2,20,1270215
1,5,9,32700
2,7,1,6617
3,9,9,171456
4,11,24,101500
...,...,...,...
509,980,36,87900
510,985,19,155000
511,987,23,135650
512,989,20,1339000


In [14]:
deposit= deposit.rename(columns={'Amount': 'Deposit Amount'})

In [15]:
total_deposit_amount = deposit.groupby('User Id').agg({'Deposit Amount':'sum'}).reset_index()

In [16]:
num_times_deposit = deposit.groupby('User Id').size().reset_index(name='no. of times deposit money')

In [17]:
deposit_data = pd.merge(total_deposit_amount, num_times_deposit, on='User Id', how='outer')

In [18]:
deposit_data

,User Id,Deposit Amount,no. of times deposit money
0,1,5000,1
1,2,567000,20
2,3,40000,4
3,4,1750,1
4,5,74100,67
...,...,...,...
910,994,1400,1
911,995,34500,5
912,996,96400,31
913,998,7400,5


In [19]:
merged = pd.merge(gameplay_data, deposit_data, on='User Id', how='outer')

In [20]:
merged

,User Id,no. of games played,Deposit Amount,no. of times deposit money
0,0,15,NaN,NaN
1,1,8,5000.0,1.0
2,2,97,567000.0,20.0
3,3,80,40000.0,4.0
4,4,5,1750.0,1.0
...,...,...,...,...
995,995,18,34500.0,5.0
996,996,474,96400.0,31.0
997,997,109,NaN,NaN
998,998,4,7400.0,5.0


In [28]:
gaming_data = pd.merge(merged, withdraw_data, on='User Id', how='outer').fillna(0)

In [29]:
gaming_data

,User Id,no. of games played,Deposit Amount,no. of times deposit money,no. of times withdrawn,Withdraw Amount
0,0,15,0.0,0.0,0.0,0.0
1,1,8,5000.0,1.0,0.0,0.0
2,2,97,567000.0,20.0,20.0,1270215.0
3,3,80,40000.0,4.0,0.0,0.0
4,4,5,1750.0,1.0,0.0,0.0
...,...,...,...,...,...,...
995,995,18,34500.0,5.0,0.0,0.0
996,996,474,96400.0,31.0,0.0,0.0
997,997,109,0.0,0.0,0.0,0.0
998,998,4,7400.0,5.0,0.0,0.0


#    

# Part A - Calculating loyalty points

# 2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 

In [30]:
gaming_data['Loyalty Points'] = (0.01 * gaming_data['Deposit Amount']) + (0.005 * gaming_data['Withdraw Amount']) + (0.001 * (max(gaming_data['no. of times deposit money']- gaming_data['no. of times withdrawn']) or 0)) + (0.2 * gaming_data['no. of games played'])

In [31]:
gaming_data

,User Id,no. of games played,Deposit Amount,no. of times deposit money,no. of times withdrawn,Withdraw Amount,Loyalty Points
0,0,15,0.0,0.0,0.0,0.0,3.215
1,1,8,5000.0,1.0,0.0,0.0,51.815
2,2,97,567000.0,20.0,20.0,1270215.0,12040.690
3,3,80,40000.0,4.0,0.0,0.0,416.215
4,4,5,1750.0,1.0,0.0,0.0,18.715
...,...,...,...,...,...,...,...
995,995,18,34500.0,5.0,0.0,0.0,348.815
996,996,474,96400.0,31.0,0.0,0.0,1059.015
997,997,109,0.0,0.0,0.0,0.0,22.015
998,998,4,7400.0,5.0,0.0,0.0,75.015


In [32]:
sorted_df = gaming_data.sort_values(by='Loyalty Points', ascending=False)

In [33]:
sorted_df

,User Id,no. of games played,Deposit Amount,no. of times deposit money,no. of times withdrawn,Withdraw Amount,Loyalty Points
634,634,24,515000.0,8.0,67.0,15737705.0,83843.540
99,99,10,1164800.0,47.0,15.0,2403141.0,23665.920
672,672,10,2158700.0,35.0,5.0,233750.0,22757.965
212,212,1,1924981.0,26.0,4.0,589850.0,22199.475
740,740,2,1738490.0,91.0,7.0,365288.0,19211.955
...,...,...,...,...,...,...,...
643,643,2,0.0,0.0,0.0,0.0,0.615
858,858,2,0.0,0.0,0.0,0.0,0.615
388,388,2,0.0,0.0,0.0,0.0,0.615
507,507,2,0.0,0.0,0.0,0.0,0.615


#                                  

# Part B - How much bonus should be allocated to leaderboard players?

# 1. Only top 50 ranked players are awarded bonus


In [69]:
top_50_loyal_players=sorted_df.head(50)

In [71]:
top_50_loyal_players = top_50_loyal_players.copy()

In [72]:
total_rewards = 50000
n=50
total_ranks = n*(n+1)/2

In [73]:
Rewards = []
for i in range(1,51):
    reward= total_rewards*(n-i+1)/total_ranks
    Rewards.append(reward)
top_50_loyal_players['Reward Distribution By Loyalty Points'] = Rewards

In [79]:
top_50_loyal_players.index += 1

In [80]:
top_50_loyal_players

,User Id,no. of games played,Deposit Amount,no. of times deposit money,no. of times withdrawn,Withdraw Amount,Loyalty Points,Reward Distribution By Loyalty Points
1,634,24,515000.0,8.0,67.0,15737705.0,83843.540,1960.784314
2,99,10,1164800.0,47.0,15.0,2403141.0,23665.920,1921.568627
3,672,10,2158700.0,35.0,5.0,233750.0,22757.965,1882.352941
4,212,1,1924981.0,26.0,4.0,589850.0,22199.475,1843.137255
5,740,2,1738490.0,91.0,7.0,365288.0,19211.955,1803.921569
6,566,183,1819175.0,53.0,3.0,185071.0,19153.920,1764.705882
7,714,6,1676300.0,34.0,0.0,0.0,16764.415,1725.490196
8,421,1557,878600.0,99.0,84.0,1269809.0,15446.660,1686.274510
9,369,37,650000.0,13.0,9.0,1586208.0,14438.655,1647.058824
10,30,13,1329000.0,51.0,1.0,152145.0,14053.540,1607.843137


#               